## Function to create a list with all the file paths within a path

In [21]:
def getFilePaths(path):
    # Function to return a list of files in a path:
    # Initialize a list for all the file names
    files = list()
    # Look for folders in the list
    for file in os.listdir(path):
        # Concatenate a path for each file
        pathfile = os.path.join(path, file)
        # Validate if the pathfile is a folder
        if os.path.isdir(pathfile):
            files = files + getFilePaths(pathfile)
        else:
            files.append(pathfile)
    return files      

## Create a dataframe with the list of paths

In [22]:
# Import dependencies
import pandas as pd, time, hashlib

In [23]:
# Path with the files to look for duplicates
path='../Photos'
# Initialize and empty dataframe with the paths of the files in the given path
files_df = pd.DataFrame(getFilePaths(path),columns=['path'])
# Initialize Columns for the dataframe 
files_df['hash'] = 0
files_df['filename'] = ''
files_df['count'] = 1
files_df['created'] = 0
files_df['modified'] = 0

# Loop to fill the dataframe columns with values
for row in range(len(files_df)):
    # Load the file
    image_file = open(files_df['path'][row],'rb').read()
    # Encode the file to a hexagsimal number
    files_df['hash'][row] = hashlib.md5(image_file).hexdigest()
    # Get the filename of the file
    files_df['filename'][row] = os.path.basename(files_df['path'][row])
    # Get the created date 
    files_df['created'][row] = pd.to_datetime(time.ctime(os.path.getctime(files_df['path'][row])))
    # Get the modified date 
    files_df['modified'][row] = pd.to_datetime(time.ctime(os.path.getmtime(files_df['path'][row])))
# Print first rows of the dataframe    
files_df.head()

,path,hash,filename,count,created,modified
0,../Photos/.DS_Store,5f27466e92caf57a930058b683ad5f7e,.DS_Store,1,2020-04-09 18:38:53,2020-04-09 18:38:53
1,../Photos/New Folder With Items 5/DSC_0030.jpg,b8cc4ead5b44bdd8e994c21287e618e8,DSC_0030.jpg,1,2020-04-09 18:38:38,2020-04-07 20:02:04
2,../Photos/New Folder With Items 5/DSC_0024.jpg,683d7efc80e910a2691880ca0ad9d059,DSC_0024.jpg,1,2020-04-09 18:38:38,2020-04-07 19:59:50
3,../Photos/New Folder With Items 5/DSC_0018.jpg,689a4e61ff572cb10ca471fb58004df1,DSC_0018.jpg,1,2020-04-09 18:38:38,2020-04-07 19:54:48
4,../Photos/New Folder With Items 5/DSC_0019.jpg,4d8c40e515d43b15a2dd6f826fdd1163,DSC_0019.jpg,1,2020-04-09 18:38:38,2020-04-07 19:55:54


In [24]:
import numpy as np

reapeated_files_df = files_df.groupby('hash').sum()
reapeated_files_df = reapeated_files_df[reapeated_files_df['count'] > 1]
pivot_df = pd.pivot_table(reapeated_files_df, values='count', index=['hash'], aggfunc=np.sum, margins=True)
pivot_df

,count
hash,
2ce0738fb3a0787942c97a0c07d25c2c,2
6bcb3dd1f6148d9c388d32953b7a030c,2
933827ccd64bd5345025069ddc79451f,4
ac30e5f00f3a848e6b174b081de267f8,2
cc5e1128d73fa665d69939e6c088da18,2
f67308e65280f43c78398f1c9da6c52d,2
All,14


In [25]:
repeated_path_df = files_df[files_df['hash'].isin(pivot_df.index.to_list())]
repeated_path_df = repeated_path_df.groupby(['hash','path']).sum()
repeated_path_df

count
hash                             path                                                     
2ce0738fb3a0787942c97a0c07d25c2c ../Photos/New Folder With Items 4/DSC_0056 copy...      1
                                 ../Photos/New Folder With Items 4/DSC_0056.jpg          1
6bcb3dd1f6148d9c388d32953b7a030c ../Photos/New Folder With Items 4/DSC_0042 copy...      1
                                 ../Photos/New Folder With Items 4/DSC_0042.jpg          1
933827ccd64bd5345025069ddc79451f ../Photos/New Folder With Items/0009845a-53ab-4...      1
                                 ../Photos/New Folder With Items/0009845a-53ab-4...      1
                                 ../Photos/New Folder With Items/0009845a-53ab-4...      1
                                 ../Photos/New Folder With Items/0009845a-53ab-4...      1
ac30e5f00f3a848e6b174b081de267f8 ../Photos/New Folder With Items 4/DSC_0040 copy...      1
                                 ../Photos/New Folder With Items 4/DSC_0040.jpg          1
cc5e1128d73fa665d69939e6c088da18 ../Photos/New Folder With Items 4/DSC_0057 copy...      1
                                 ../Photos/New Folder With Items 4/DSC_0057.jpg          1
f67308e65280f43c78398f1c9da6c52d ../Photos/New Folder With Items 4/DSC_0041 copy...      1
                                 ../Photos/New Folder With Items 4/DSC_0041.jpg          1